# Testing the class 'GeneralizedAlgebraicKnot'

## Setup

In [ ]:
%matplotlib inline
%load_ext pycodestyle_magic

In [ ]:
# display full output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# Import your preparse magic (assuming it's saved in a utility script or redefined here)
from IPython.core.magic import register_line_magic
import path
from gaknot.utility import import_sage

@register_line_magic
def preparse(line):
    """
    Custom magic to preparse a sage file using the gaknot utility logic.
    Usage: %preparse signature
    """
    # Assuming 'gaknot' is in the current directory or python path
    import os
    package_name = 'gaknot'
    # Get the path to the package
    path = os.path.dirname(os.path.abspath(package_name))
    
    try:
        import_sage(line.strip(), package=package_name, path=path)
        print(f"Successfully preparsed and reloaded: {line}")
    except Exception as e:
        print(f"Error during preparse: {e}")

# Registering the magic
# del preparse # Optional: allows re-running this cell to update the magic
register_line_magic(preparse)

In [ ]:
# Preparse the new class file
%preparse gaknot
from gaknot.gaknot import GeneralizedAlgebraicKnot

## Tests

### Testing basic functionality

In [ ]:
# Define a valid connected sum: T(2,3) # -T(2,3; 6,5)
desc1 = [(1, [(2, 3)]), (-1, [(2, 3), (6, 5)])]

knot1 = GeneralizedAlgebraicKnot(desc1)

# Test __str__
print("Human readable string:")
print(knot1)

# Test __repr__
print("\nDeveloper representation:")
knot1

### Testing validation and error handling

In [ ]:
invalid_descriptions = [
    # 1. Not coprime
    ("Coprimality error", [(1, [(2, 4)])]),
    # 2. Not positive integers
    ("Positivity error", [(1, [(1, 3)])]),
    # 3. Bad sign
    ("Sign error", [(2, [(2, 3)])]),
    # 4. Bad structure (missing list for knot desc)
    ("Structure error", [(1, (2, 3))]) 
]

for error_name, bad_desc in invalid_descriptions:
    print(f"--- Testing {error_name} ---")
    try:
        GeneralizedAlgebraicKnot(bad_desc)
        print("FAIL: Exception was not raised!")
    except (ValueError, TypeError) as e:
        print(f"Caught expected error: {e}\n")

### Testing algebraic operations

In [ ]:
# Define another knot: T(3,4)
desc1 = [(1, [(2, 3)]), (-1, [(2, 3), (6, 5)])]

knot_1 = GeneralizedAlgebraicKnot(desc1)
knot_2 = GeneralizedAlgebraicKnot([(1, [(3, 4)])])

print(type(knot_1))
print(type(knot_2))

# Test connected sum (__add__)
sum_knot = knot_1 + knot_2
print("Connected Sum:")
print(sum_knot)

# Test concordance inverse (__neg__)
inverse_knot = -knot_1
print("\nConcordance Inverse:")
print(inverse_knot)

# Test combination
complex_knot = knot_1 + (-knot_2)
print("\nComplex Combination (knot1 - knot2):")
print(complex_knot)

In [ ]:
# 1. Define the slice knot T(2,3) # -T(2,3)
desc = [(1, [(2, 3)]), (-1, [(2, 3)])]
slice_knot = GeneralizedAlgebraicKnot(desc)

# 2. Compute the signature directly from the object
sig_func = slice_knot.signature()

# 3. Verify it evaluates to 0 (since it's a slice knot)
print(f"Is the signature zero everywhere? {sig_func.is_zero_everywhere()}")

In [ ]:
# 4. Plot it just to be sure
from gaknot.signature import SignaturePloter
SignaturePloter.plot(sig_func, title=f"Signature of {slice_knot}")

In [ ]:
# Define the knot T(2,3;5,2) # T(3,2) # T(5,3) # -T(6,5)
desc = [
    (1, [(2,3), (5,2)]),
    (1, [(3,2)]),
    (1, [(5,3)]),
    (-1, [(6,5)])
]

alg_slice_knot = GeneralizedAlgebraicKnot(desc)

print(alg_slice_knot)

In [ ]:
# compute the LT_signature
sig_f = alg_slice_knot.signature()

# 3. Verify if it evaluates to 0 (since it's an algebraically slice knot)
print(f"Is the signature zero everywhere? {sig_func.is_zero_everywhere()}")

In [ ]:
# Plot the signature function
from gaknot.signature import SignaturePloter
SignaturePloter.plot(sig_func, title=f"Signature of {alg_slice_knot}")

### Testing Alexander polynomial

In [ ]:
# 1. Test for the Trefoil knot T(2,3)
desc_trefoil = [(1, [(2, 3)])]
knot_trefoil = GeneralizedAlgebraicKnot(desc_trefoil)

poly_trefoil = knot_trefoil.alexander_polynomial()
print(f"Alexander polynomial of {knot_trefoil}:")
print(poly_trefoil)
print("Expected: t^2 - t + 1\n")

In [ ]:
# 2. Test for the connected sum T(2,3) # T(2,3)
# The polynomial should be the square of the Trefoil's polynomial
desc_sum = [(1, [(2, 3)]), (1, [(2, 3)])]
knot_sum = GeneralizedAlgebraicKnot(desc_sum)

poly_sum = knot_sum.alexander_polynomial()
print(f"Alexander polynomial of {knot_sum}:")
print(poly_sum)
print("Expected: t^4 - 2*t^3 + 3*t^2 - 2*t + 1\n")

In [ ]:
# 3. Test for an iterated torus knot T(2,3; 2,5)
# Using the cabling formula: Delta_{2,3; 2,5}(t) = Delta_{2,5}(t) * Delta_{2,3}(t^2)
# Delta_{2,5}(t) = t^4 - t^3 + t^2 - t + 1
# Delta_{2,3}(t^2) = t^4 - t^2 + 1
desc_cable = [(1, [(2, 3), (2, 5)])]
knot_cable = GeneralizedAlgebraicKnot(desc_cable)

poly_cable = knot_cable.alexander_polynomial()
print(f"Alexander polynomial of {knot_cable}:")
print(poly_cable)
print("Expected: t^8 - t^7 + t^5 - t^4 + t^3 - t + 1\n")

### Testing container-like behavior

In [ ]:
# Define a knot consisting of 3 summands: T(2,3) # T(3,4) # -T(2,5)
desc_container = [
    (1, [(2, 3)]),
    (1, [(3, 4)]),
    (-1, [(2, 5)])
]
knot_container = GeneralizedAlgebraicKnot(desc_container)

In [ ]:
# 1. Testing __len__
print(f"Number of summands (len): {len(knot_container)}")
assert len(knot_container) == 3

In [ ]:
# 2. Testing __getitem__ with integer indexing
print(f"First summand (knot[0]): {knot_container[0]}")
print(f"Last summand (knot[-1]): {knot_container[-1]}")

In [ ]:
# 3. Testing __getitem__ with slicing
print(f"Sliced summands (knot[0:2]): {knot_container[0:2]}")

In [ ]:
# 4. Testing iteration (relies on __getitem__ handling indices properly)
print(f"Iterating over the summands of the knot {knot_container}:")
for i, summand in enumerate(knot_container):
            print(f"\tSummand {i}: {summand}")

In [ ]:
# 5. Testing exception handling (IndexError)
print("Testing out-of-bounds access:")
try:
    knot_container[10]
    print("ERROR: IndexError was not raised!")
except IndexError as e:
    print(f"\tCaught expected exception: {e}")

### Testing verification of the knot type

In [ ]:
knot_pos_torus = GeneralizedAlgebraicKnot([(1, [(2, 3)])])
knot_neg_torus = GeneralizedAlgebraicKnot([(-1, [(2, 3)])])
knot_pos_iterated = GeneralizedAlgebraicKnot([(1, [(2, 3), (2, 5)])])
knot_sum = GeneralizedAlgebraicKnot([(1, [(2, 3)]), (1, [(3, 4)])])

In [ ]:
print("Testing T(2,3):")
assert knot_pos_torus.is_positive_torus_knot() == True
assert knot_pos_torus.is_iterated_torus_knot() == True
assert knot_pos_torus.is_negative_torus_knot() == False

In [ ]:
print("Testing -T(2,3):")
assert knot_neg_torus.is_negative_torus_knot() == True
assert knot_neg_torus.is_neg_iterated_torus_knot() == True
assert knot_neg_torus.is_positive_torus_knot() == False

In [ ]:
print("Testing T(2,3; 2,5):")
assert knot_pos_iterated.is_iterated_torus_knot() == True
assert knot_pos_iterated.is_positive_torus_knot() == False # It's iterated, not basic

In [ ]:
print("Testing T(2,3) # T(3,4):")
assert knot_sum.is_positive_torus_knot() == False # It's a connected sum
assert knot_sum.is_iterated_torus_knot() == False 